In [1]:
import anthropic
import base64
import json
import pandas as pd

In [2]:
# pip install anthropic

In [3]:
# get api_key from file (it is saved in format "api_key: 123456 #comment")
with open('api_key.txt') as fh:
    api_key = fh.readline() \
                     .split(':')[1].split()[0]


client = anthropic.Anthropic(api_key = api_key)

# api_key

In [9]:
def upload_image_from_disk(path_to_image):
    # read the image file from disk and encode it to base64 string
    with open(path_to_image, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

    return encoded_image


path_to_image = 'input\\Life expectancy in mice.png'
encoded_image = upload_image_from_disk(path_to_image)
# encoded_image[-100:]

<br>
<br>

In [93]:
def prompt_Claude(client,
                  system = "You are an expert in Kaplan-Meier curves.",
                  prompt = "",
                  encoded_image = ""):

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        system=system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": encoded_image
                        }
                    }
                ]
            }
        ]
    )

    return response


system = "You are an expert in Kaplan-Meier curves."
prompt = "That is a chart of life expectancy for 3 groups of mice. For each group, what is the age at which survival reaches 15%? " + \
          "Your answer must be only unrounded values in the form of a Python dictionary."

df = pd.DataFrame()

number_tests = 8
for count in range(1, number_tests+1):
    print(f"{count}. ", end='')
    
    response = prompt_Claude(client=client, system=system, prompt=prompt, encoded_image=encoded_image)
    response_text = response.content[0].text
    print(response_text, '-', len(response_text))
    
    dd = json.loads(response_text.replace('\'', '"'))
    df_current = pd.DataFrame(data=dd, index=[count])
    df = pd.concat([df, df_current], axis='index')


df.to_csv('output\\result of experiment.csv')

df